# Replication

I cant find model weights, only the layers of the final quanitzed model 

Instead I will replicate the architecture and the way training data was split etc

Training Input is everything except classification

Data is scaled in the following ways:
- pT ([:,:,0]) Is z-scaled (x - mean)/std : They say they bring all to O(1) but not how so this seems reasonable
- eta and phi are tanh scaled : tanh(x / x_b) where x_b is the max(abs(x)) for non padded entries : This is only done before the MSE loss, implementing this for KL is very non trivial 

Models are constructed as classes with the encoder and decoder created as submodels

So if we have a full model say CNN_VAE we access the encoder with either CNN_VAE.layers[0] or CNN_VAE.get_layer("cnn_encoder"), but check what the actual name is, the submodel extracted then has the same methods available

The MSE loss and masked MSE loss is handled as functions that require a partial to work due to the way tensorflow works

The beta parameter managing the balance between MSE and KL is not defined in the paper -> A number of values are trialed and the best is selected -> Note that this is non exhaustive of a search and only done to order of magnitude not to exact value

Also note that if you run this and an error occurs during any step whatsoever you will likely have to restart the kernel as I am now convinced that this causes Memory leaks either due to jupyter loosing the reference or tensorflow handling the reference in an odd fashion -> I would like to blame tensorflow rather than jupyter

Misc Notes:

When you inevitably run into nan errors dont bother printing things use tf.debugging.enable_check_numerics() instead, this will tell the compiler to raise an exception on nan and inf with a full backtrace on which layer this occured, look for a long / seperated string that contains layer and operation names it essentially gives you the location, if its not clear copy paste with the model architecture to chatGPT it will usually identify the location (allthough it really likes quoting the obvious reasons nan may arrise that have nothing to do with the current problem)

For debugging print works when not compiled, tf.print works when compiled, i always just do both and that works

In [1]:
# Loss Functions --- Reasoning written out where data is loaded -> Partial needs to be called on these 
import tensorflow as tf
@tf.function
def scale_for_MSE(y_true, y_pred, eta_b, phi_b):
    """
    Loss scaling helper
    """
    # Extract
    eta_true = y_true[:, :, 1]
    phi_true = y_true[:, :, 2]
    eta_pred = y_pred[:, :, 1]
    phi_pred = y_pred[:, :, 2]
    # Apply tanh scaling
    eta_true_scaled = tf.tanh(eta_true / eta_b)
    phi_true_scaled = tf.tanh(phi_true / phi_b)
    eta_pred_scaled = tf.tanh(eta_pred / eta_b)
    phi_pred_scaled = tf.tanh(phi_pred / phi_b)
    # Replace and hope compilation makes this efficient
    y_true_scaled = tf.concat([
        y_true[:, :, :1],                           
        eta_true_scaled[:, :, tf.newaxis],          
        phi_true_scaled[:, :, tf.newaxis],          
        y_true[:, :, 3:],                           
    ], axis=-1)

    y_pred_scaled = tf.concat([
        y_pred[:, :, :1],
        eta_pred_scaled[:, :, tf.newaxis],
        phi_pred_scaled[:, :, tf.newaxis],
        y_pred[:, :, 3:],
    ], axis=-1)
    return y_true_scaled, y_pred_scaled

@tf.function
def full_MSE(y_true, y_pred, eta_b, phi_b):
    "Full Loss Function overwrite"
    y_true_scaled, y_pred_scaled = scale_for_MSE(y_true, y_pred, eta_b, phi_b)
    # Compute MSE 
    mse = tf.reduce_mean(
        tf.reduce_sum(tf.square(y_true_scaled - y_pred_scaled), axis=(1, 2))
    )
    
    return mse

def masked_mse(y_true_og, y_pred_og, eta_b, phi_b, epsilon=1e-12):
    """
    Computes MSE per event, ignoring any zero-padded rows.
    Assumes y_true, y_pred each have shape (batch, n_features)
    or possibly (batch, H, W, C). Adjust logic as needed.
    """
    y_true, y_pred = scale_for_MSE(y_true_og, y_pred_og, eta_b, phi_b)
    # Sum over feature-dims to see whether row is all zeros
    row_sum = tf.reduce_sum(tf.abs(y_true), axis=-1)  # shape = (batch,)
    # A mask: 1 if non-padded, 0 if padded
    mask = tf.cast(row_sum > epsilon, tf.float32)
    # Squared error, summation over features
    sq_error = tf.reduce_sum(tf.square(y_true - y_pred), axis=-1)  # shape=(batch,)
    # Apply mask -> I hate this
    sq_error_masked = sq_error * mask
    return tf.reduce_sum(sq_error_masked) / tf.reduce_sum(mask)

def tmp():
    """For VAE implementation a lot of code is taken from 
    https://keras.io/examples/generative/vae/
    """
    import numpy as np
    import tensorflow as tf
    import keras
    from keras import ops
    from keras import layers
    class Sampling(layers.Layer):
        """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

        def __init__(self, **kwargs):
            super().__init__(**kwargs)
            self.seed_generator = keras.random.SeedGenerator(1337)

        def call(self, inputs):
            z_mean, z_log_var = inputs
            batch = ops.shape(z_mean)[0]
            dim = ops.shape(z_mean)[1]
            epsilon = keras.random.normal(shape=(batch, dim), seed=self.seed_generator)
            return z_mean + ops.exp(0.5 * z_log_var) * epsilon


    return Sampling
Sampling = tmp()

2025-03-13 09:08:45.428548: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-13 09:08:45.819145: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-13 09:08:47.385736: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
from tensorflow.keras import layers, Model

def dnn_reshape():
    """Input Space for DNN is flattened"""
    return layers.Flatten()

def dnn_encoder(input_dim = 57, latent_dim = 3, name='DNN_Encoder', vae=False):
    inputs = tf.keras.Input(shape=(19,3,), name="encoder_in")
    x = inputs#layers.BatchNormalization()(inputs)
    x = layers.Flatten()(x)
    
    x = layers.Dense(32)(x) # Activation is None By Default
    x = layers.LeakyReLU()(x)
    x = layers.Dense(16)(x)
    x = layers.LeakyReLU()(x)

    if vae:
        z_mean = layers.Dense(latent_dim, name="z_mean")(x)
        # Kernel initializer as this term is exponentiated and default init makes the exp -> ooo
        z_log_var = layers.Dense(latent_dim, name="z_log_var", kernel_initializer="zeros")(x)
        z = Sampling()([z_mean, z_log_var])
        return Model(inputs, [z_mean, z_log_var, z], name=name)

    latent = layers.Dense(latent_dim, name="latent")(x)

    return Model(inputs, latent, name=name)

def dnn_decoder(output_dim=57, latent_dim=3, name='DNN_Decoder'):
    """
    BN between Fully connected and activation, last doesnt have activation & BN
    """
    latent_in = tf.keras.Input(shape=(latent_dim,), name="decoder_in")

    y = layers.Dense(16)(latent_in)
    y = layers.BatchNormalization()(y)
    y = layers.LeakyReLU()(y)

    y = layers.Dense(32)(y)
    y = layers.BatchNormalization()(y)
    y = layers.LeakyReLU()(y)

    outputs = layers.Dense(output_dim, name="reconstruction")(y)

    # And reshape so that the loss works right
    outputs = layers.Reshape((19,3,))(outputs)

    return Model(latent_in, outputs, name=name)

dnn_encoder().summary()
dnn_decoder().summary()

Model: "DNN_Encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder_in (InputLayer)         │ (None, 19, 3)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 57)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         1,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ latent (Dense)                  │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,435 (9.51 KB)

 Trainable params: 2,435 (9.51 KB)

 Non-trainable params: 0 (0.00 B)

Model: "DNN_Decoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ decoder_in (InputLayer)         │ (None, 3)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reconstruction (Dense)          │ (None, 57)             │         1,881 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 19, 3)          │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,681 (10.47 KB)

 Trainable params: 2,585 (10.10 KB)

 Non-trainable params: 96 (384.00 B)

In [3]:
@tf.keras.utils.register_keras_serializable()
class DNN_AE(tf.keras.Model):
    """
    AE = Encoder -> Decoder, no sampling.
    """
    def __init__(self, input_dim=57, latent_dim=3):
        super().__init__()
        self.encoder = dnn_encoder(input_dim=input_dim, latent_dim=latent_dim)
        self.decoder = dnn_decoder(output_dim=input_dim, latent_dim=latent_dim)

    def call(self, x):
        z = self.encoder(x)            # shape=(batch, latent_dim)
        x_recon = self.decoder(z)      # shape=(batch, input_dim)
        return x_recon


@tf.keras.utils.register_keras_serializable()
class DNN_VAE(tf.keras.Model):
    def __init__(self, eta_b, phi_b, input_dim=57, latent_dim=3, beta=1.0, **kwargs):
        super().__init__(**kwargs)
        self.encoder = dnn_encoder(input_dim=input_dim, latent_dim=latent_dim, vae=True)
        self.decoder = dnn_decoder(output_dim=input_dim, latent_dim=latent_dim)
        self.beta = beta
        self.eta_b = eta_b
        self.phi_b = phi_b
        
        self.total_loss_tracker = tf.keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = tf.keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = tf.keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)

            reconstruction_loss = tf.reduce_mean(
                masked_mse(data, reconstruction, self.eta_b, self.phi_b)
            )

            # ✅ KL divergence
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))

            # ✅ Total loss with β weighting
            total_loss = (1.0 - self.beta) * reconstruction_loss + self.beta * kl_loss

        # ✅ Backpropagation
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

        # ✅ Update metrics
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)

        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }
    
    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstruction = self.decoder(z)
        return reconstruction

In [4]:
def build_cnn_encoder(name='cnn_encoder', vae=False):
    """
    Build CNN Encoder
    vae : Variational Autoencoder varitation (second final layer)
    """

    inputs = tf.keras.Input(shape=(19,3,1), name="cnn_encoder_input")
    #      Zeropad to (20,3,1)  - Syntax padding = ((top, bottom), (left, right))
    x = layers.ZeroPadding2D(padding=((1,0),(0,0)))(inputs)
    x = layers.BatchNormalization()(x)
    #      Block 1  -> Padding valid means no padding
    x = layers.Conv2D(16, kernel_size=(3,3), strides=(1,1), padding='valid', use_bias=False)(x)
    x = layers.ReLU()(x)
    x = layers.AveragePooling2D(pool_size=(3,1), strides=(3,1))(x)
    #      Block 2
    x = layers.Conv2D(32, kernel_size=(3,1), strides=(1,1), padding='valid', use_bias=False)(x)
    x = layers.ReLU()(x)
    x = layers.AveragePooling2D(pool_size=(3,1), strides=(3,1))(x)
    x = layers.Flatten()(x)
    #      Block 3 - started
    if vae:
        latent_mean = layers.Dense(8, activation=None, name="latent_mean")(x)
        latent_log_var = layers.Dense(8, activation=None, name="latent_log_var", kernel_initializer='zeros')(x)
        z = Sampling()([latent_mean, latent_log_var])
        return tf.keras.Model(inputs, [latent_mean, latent_log_var, z], name=name)

    latent = layers.Dense(8, activation=None, name="latent")(x)
    return tf.keras.Model(inputs, latent, name=name)


def build_cnn_decoder(name='cnn_decoder'):
    """
    CNN decoder from the paper's bottom figure. 
    Takes a latent dimension (8) -> Dense -> Reshape -> Conv2DTranspose or Upsampling + Conv2D 
    to go back to shape (20,3,1) then maybe slice off padding if you want 19,3,1 final.
    """
    #      Block 3 - remainder > Forming encoder input 
    latent_inputs = tf.keras.Input(shape=(8,), name="cnn_decoder_input")
    y = layers.Dense(64, activation=None)(latent_inputs) 
    y = layers.ReLU()(y)
    y = layers.Reshape((2,1,32))(y)
    #      Block 4
    y = layers.Conv2D(32, kernel_size=(3,1), strides=(1,1), padding='same')(y)
    y = layers.ReLU()(y)
    y = layers.UpSampling2D(size=(3,1))(y)
    y = layers.ZeroPadding2D(padding=((0,0),(1,1)))(y)
    #      Block 5
    y = layers.Conv2D(16, kernel_size=(3,1), strides=(1,1), padding='same')(y)
    y = layers.ReLU()(y)
    y = layers.UpSampling2D(size=(3,1))(y)
    y = layers.ZeroPadding2D(padding=((1,0),(0,0)))(y)
    #      Output 
    output = layers.Conv2D(1, kernel_size=(3,3), strides=(1,1), padding='same')(y)

    return tf.keras.Model(latent_inputs, output, name=name)

build_cnn_encoder().summary()
build_cnn_decoder().summary()

Model: "cnn_encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ cnn_encoder_input (InputLayer)  │ (None, 19, 3, 1)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ zero_padding2d (ZeroPadding2D)  │ (None, 20, 3, 1)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 20, 3, 1)       │             4 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 18, 1, 16)      │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 18, 1, 16)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 6, 1, 16)       │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 4, 1, 32)       │         1,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 4, 1, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 1, 1, 32)       │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ latent (Dense)                  │ (None, 8)              │           264 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,948 (7.61 KB)

 Trainable params: 1,946 (7.60 KB)

 Non-trainable params: 2 (8.00 B)

Model: "cnn_decoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ cnn_decoder_input (InputLayer)  │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ (None, 2, 1, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 2, 1, 32)       │         3,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_3 (ReLU)                  │ (None, 2, 1, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 6, 1, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ zero_padding2d_1                │ (None, 6, 3, 32)       │             0 │
│ (ZeroPadding2D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 6, 3, 16)       │         1,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_4 (ReLU)                  │ (None, 6, 3, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 18, 3, 16)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ zero_padding2d_2                │ (None, 19, 3, 16)      │             0 │
│ (ZeroPadding2D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 19, 3, 1)       │           145 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,377 (21.00 KB)

 Trainable params: 5,377 (21.00 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
@tf.keras.utils.register_keras_serializable()
class CNN_AE(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.encoder = build_cnn_encoder(name="cnn_encoder")
        self.decoder = build_cnn_decoder(name="cnn_decoder")

    def call(self, x):
        z = self.encoder(x)
        recon = self.decoder(z)
        return recon


@tf.keras.utils.register_keras_serializable()
class CNN_VAE(tf.keras.Model):
    def __init__(self,eta_b, phi_b, input_shape=(19, 3, 1), latent_dim=3, beta=1.0, **kwargs):
        super().__init__(**kwargs)
        self.encoder = build_cnn_encoder(vae=True)
        self.decoder = build_cnn_decoder()
        self.beta = beta
        self.eta_b = eta_b
        self.phi_b = phi_b
        
        self.total_loss_tracker = tf.keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = tf.keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = tf.keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        """
        Custom train step
        We do the normal thing
        But also add the KL divergence term        
        """
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)

            # MSE over all spatial and channel dimensions
            #reconstruction_loss = tf.reduce_mean(
            #    tf.reduce_sum(tf.square(data - reconstruction), axis=(1, 2, 3))
            #)
            reconstruction_loss = full_MSE(data , reconstruction[:,:,:,0], self.eta_b, self.phi_b)

            # KL divergence term
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))

            # Total loss with β weighting
            total_loss = (1.0 - self.beta) * reconstruction_loss + self.beta * kl_loss

        # Backprop
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

        # Update
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)

        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }
    
    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstruction = self.decoder(z)
        return reconstruction

## Load Data + Define training Loops

In [6]:
import h5py
import numpy as np
# If you use the dockerfile this should just work 
dset = h5py.File('/Code/Dataset/background_for_training.h5', 'r')
dset = {key: dset[key][()] for key in dset.keys()}
"""
Contains keys:
    Particles_Classes : 4 classes of Particles
    Particles_Names : Names of the Particles
    Particles : The data (n, 19,4)
        19 : Indexes are
            - 0 : Missing Transverse Energy
            - 1:4 Up to 4 electrons
            - 4:8 Up to 4 muons
            - 8-18 Up to 10 jets
        Subdimension 4 by idx:
            - 0 : pT (transverse momentum)
            - 1 : eta (pseudorapidity)
            - 2 : phi (azimuthal angle)
            - 3 : class (0=Nothing, 1=Met,2=electron,3=muon,4=jet)
And when something doesnt make sense (ie [:,0,1:4]) its just zero 
"""
data = dset['Particles']
del dset
# Do z score norm to aid in training : They dont specify how they made O(1) : And I assume they mean across all defined objects
detected_bmap = (data[:,:,3] != 0) # Select defined entries
mean_pt = tf.reduce_mean(data[detected_bmap, 0])
std_pt = tf.math.reduce_std(data[detected_bmap, 0])
data[:,:,0] = ((data[:,:,0] - mean_pt) / std_pt)

"""
They mention in the paper:
To account for physical boundaries of η and φ, for those features a re-scaled tanh activation is used in the loss computation.

So I assume this means we need to find the extent of the parameters (call it b ) and do tanh(x/b) prior to the loss function, 
I am not sure how one would go about implementing this for KL loss so Im going to do MSE only 
"""
eta_b = np.maximum(np.max(data[detected_bmap, 1]), np.abs(np.min(data[detected_bmap, 1]))).astype("float32")
phi_b = np.maximum(np.max(data[detected_bmap, 2]), np.abs(np.min(data[detected_bmap, 2]))).astype("float32")

In [7]:
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

train_split = int(0.5*len(data))
val_split = int((0.4+0.5)*len(data))

n = np.arange(len(data)) 
np.random.shuffle(n)
# Do splitting - Dont grab class indices
train = data[n[:train_split],          :, :3]
val   = data[n[train_split:val_split], :, :3]
test  = data[n[val_split:],            :, :3]

In [15]:
from functools import partial
def train_dnn_ae(x_train, x_val, epochs=100, batch_size=1024):
    """
    Create and train
    """
    # Build 
    dnn_ae = DNN_AE(
        input_dim=57, latent_dim=3
        )

    # Compile with Adam and masked MSE
    loss_func = tf.function(partial(masked_mse, eta_b=eta_b, phi_b=phi_b))
    dnn_ae.compile(
        optimizer=tf.keras.optimizers.Adam(), 
        loss=loss_func
    )
    # Make Callback
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=10, restore_best_weights=True # restore???
        )
    ]
    # Fit 
    dnn_ae.fit(
        x_train, x_train,
        validation_data=(x_val, x_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        verbose=1
    )
    return dnn_ae

def train_dnn_vae(x_train, x_val, epochs=100, batch_size=1024, beta=1.0):
    """
    Create and train
    """
    # Build 
    dnn_vae = DNN_VAE(eta_b=eta_b, phi_b=phi_b, input_dim=57, latent_dim=3, beta=beta)

    # Compile with Adam and masked MSE
    loss_func = tf.function(partial(masked_mse, eta_b=eta_b, phi_b=phi_b))
    dnn_vae.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, clipnorm=1.0), 
        loss=loss_func
    )
    # Make Callback
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=10, restore_best_weights=True # restore???
        )
    ]
    # Fit 
    dnn_vae.fit(
        x_train, # It doesnt have y apparently its passed to train_step as (x, y) and then autoresolved to be *(x, y) for val it does need two I do not quite get why
        validation_data=(x_val,x_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        verbose=1
    )
    return dnn_vae

In [17]:
from functools import partial

def train_cnn_ae(x_train, x_val, epochs=100, batch_size=1024):
    """
    Create and train
    """
    cnn_ae_model = CNN_AE(
        # No Args
    )
    loss_func = tf.function(partial(full_MSE, eta_b=eta_b, phi_b=phi_b))
    cnn_ae_model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=loss_func 
    )
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=10, restore_best_weights=True
        )
    ]
    cnn_ae_model.fit(
        x_train, x_train,
        validation_data=(x_val, x_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks
    )
    return cnn_ae_model

def train_cnn_vae(x_train, x_val, epochs=100, batch_size=1024, beta=1.0):
    """
    Create and train
    """
    cnn_vae_model = CNN_VAE( eta_b=eta_b, phi_b=phi_b, beta=beta )
    cnn_vae_model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss='mse' # I think this is just ignored since we have a custom train loop
    )

    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=10, restore_best_weights=True
        )
    ]
    cnn_vae_model.fit(
        x_train,
        validation_data=(x_val, x_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks
    )

    return cnn_vae_model

# Actual Training 

Note that there is a lot of duplicate code as the kernel often doesnt survive past more than one model trainings

Also the saving

It wants me to implement a get_config method for saving the model, I dont want to do that as this means it will also want me to make a load config and the probably 20 other things

I just create a new uncompiled model copy the weigths over save that (and we only really need it to save those anyway) and that should be directly loadable, also it doesnt like hdf5 for saving anymore

In [11]:
import os 
path = '/Code/Replicate/Models'
if not os.path.exists(path):
    os.makedirs(path)
epochs = 100
batch_size = 1024

dnn_ae  = train_dnn_ae( train, val, epochs=epochs, batch_size=batch_size)
# This keeps failing
#dnn_ae.save(f'{path}/full_dnn_ae.keras')
tmp = DNN_AE(
        input_dim=57, latent_dim=3
        )
tmp.set_weights(dnn_ae.get_weights())
tmp.save(f'{path}/full_dnn_ae.keras')

Epoch 1/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 92s 14ms/step - loss: 0.1535 - val_loss: 0.0763
Epoch 2/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 36s 5ms/step - loss: 0.0741 - val_loss: 0.0664
Epoch 3/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 34s 5ms/step - loss: 0.0658 - val_loss: 0.0643
Epoch 4/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 36s 5ms/step - loss: 0.0640 - val_loss: 0.0630
Epoch 5/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 34s 5ms/step - loss: 0.0628 - val_loss: 0.0625
Epoch 6/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 35s 5ms/step - loss: 0.0618 - val_loss: 0.0610
Epoch 7/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 34s 5ms/step - loss: 0.0611 - val_loss: 0.0624
Epoch 8/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 32s 5ms/step - loss: 0.0606 - val_loss: 0.0602
Epoch 9/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 32s 5ms/step - loss: 0.0603 - val_loss: 0.0599
Epoch 10/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 32s 5ms/step - loss: 0.0599 - val_loss: 0.0599
Epoch 11/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 34s 5ms/step - loss: 0.0597 - val_loss: 0.0594
Epoch 1

TypeError: Cannot serialize object <tensorflow.python.eager.polymorphic_function.polymorphic_function.Function object at 0x78595045e190> of type <class 'tensorflow.python.eager.polymorphic_function.polymorphic_function.Function'>. To be serializable, a class must implement the `get_config()` method.

In [ ]:
import os 
path = '/Code/Replicate/Models'
if not os.path.exists(path):
    os.makedirs(path)
epochs = 100
batch_size = 1024

cnn_ae  = train_cnn_ae( train, val, epochs=epochs, batch_size=batch_size)

#cnn_ae.save(f'{path}/full_cnn_ae.keras')
tmp = CNN_AE(
        )
tmp.set_weights(cnn_ae.get_weights())
tmp.save(f'{path}/full_cnn_ae.keras')

Epoch 1/100
   6/6569 ━━━━━━━━━━━━━━━━━━━━ 1:06 10ms/step - loss: 38.3073 

2025-03-13 09:09:28.435055: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT32 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


6569/6569 ━━━━━━━━━━━━━━━━━━━━ 72s 11ms/step - loss: 3.5095 - val_loss: 1.4083
Epoch 2/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 70s 11ms/step - loss: 1.3971 - val_loss: 1.3618
Epoch 3/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 69s 10ms/step - loss: 1.3478 - val_loss: 1.1569
Epoch 4/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 71s 11ms/step - loss: 1.1308 - val_loss: 1.0258
Epoch 5/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 66s 10ms/step - loss: 1.0170 - val_loss: 0.9727
Epoch 6/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 68s 10ms/step - loss: 0.9451 - val_loss: 0.8951
Epoch 7/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 70s 11ms/step - loss: 0.8884 - val_loss: 0.8683
Epoch 8/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 62s 9ms/step - loss: 0.8678 - val_loss: 0.8579
Epoch 9/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 70s 11ms/step - loss: 0.8570 - val_loss: 0.8525
Epoch 10/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 63s 10ms/step - loss: 0.8498 - val_loss: 0.8444
Epoch 11/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 57s 9ms/step - loss: 0.8445 - val_loss: 0.8414
Epoch 12/10

TypeError: CNN_AE.__init__() got an unexpected keyword argument 'input_dim'

# Beta testing

In [29]:
import os 
# Iterate e-3 to e0 
for i in np.logspace(-3, 0, 4):
    path = '/Code/Replicate/Models'
    if not os.path.exists(path):
        os.makedirs(path)
    epochs = 100
    batch_size = 1024

    dnn_vae = train_dnn_vae(train, val, epochs=epochs, batch_size=batch_size, beta=i)

    #dnn_vae.save(f'{path}/full_dnn_vae_beta{i}.keras')
    tmp = DNN_VAE(eta_b=eta_b, phi_b=phi_b, input_dim=57, latent_dim=3, beta=i
            )
    tmp.set_weights(dnn_vae.get_weights())
    tmp.save(f'{path}/full_dnn_vae_beta{i}.keras')

    

Epoch 1/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 34s 5ms/step - kl_loss: 801.7324 - loss: 28.2326 - reconstruction_loss: 27.4586 - val_kl_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_total_loss: 0.0000e+00
Epoch 2/100
  26/6569 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - kl_loss: 7.8346 - loss: 0.0820 - reconstruction_loss: 0.0742    

/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: kl_loss,loss,reconstruction_loss,val_kl_loss,val_reconstruction_loss,val_total_loss
  current = self.get_monitor_value(logs)


6569/6569 ━━━━━━━━━━━━━━━━━━━━ 29s 4ms/step - kl_loss: 7.9456 - loss: 0.0812 - reconstruction_loss: 0.0734 - val_kl_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_total_loss: 0.0000e+00
Epoch 3/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 30s 5ms/step - kl_loss: 8.1541 - loss: 0.0754 - reconstruction_loss: 0.0673 - val_kl_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_total_loss: 0.0000e+00
Epoch 4/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 31s 5ms/step - kl_loss: 8.2994 - loss: 0.0731 - reconstruction_loss: 0.0649 - val_kl_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_total_loss: 0.0000e+00
Epoch 5/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 30s 5ms/step - kl_loss: 8.3520 - loss: 0.0714 - reconstruction_loss: 0.0631 - val_kl_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_total_loss: 0.0000e+00
Epoch 6/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 31s 5ms/step - kl_loss: 8.4330 - loss: 0.0701 - reconstruction_loss: 0.0617 - val_kl_loss: 0.0000e+00 - val_reconstructio

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_api.py:107: UserWarning: You are saving a model that has not yet been built. It might not contain any weights yet. Consider building the model first by calling it on some data.
  return saving_lib.save_model(model, filepath)


Epoch 1/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 39s 6ms/step - kl_loss: 4.4773 - loss: 0.2304 - reconstruction_loss: 0.1875 - val_kl_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_total_loss: 0.0000e+00
Epoch 2/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 31s 5ms/step - kl_loss: 3.7308 - loss: 0.1278 - reconstruction_loss: 0.0914 - val_kl_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_total_loss: 0.0000e+00
Epoch 3/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 31s 5ms/step - kl_loss: 3.9671 - loss: 0.1235 - reconstruction_loss: 0.0847 - val_kl_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_total_loss: 0.0000e+00
Epoch 4/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 32s 5ms/step - kl_loss: 4.0934 - loss: 0.1210 - reconstruction_loss: 0.0809 - val_kl_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_total_loss: 0.0000e+00
Epoch 5/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 34s 5ms/step - kl_loss: 4.1181 - loss: 0.1203 - reconstruction_loss: 0.0800 - val_kl_loss: 0.0000e+00 - val_r

In [20]:
import os 
# Iterate e-3 to e0
for i in np.logspace(-3, 0, 4):
    path = '/Code/Replicate/Models'
    if not os.path.exists(path):
        os.makedirs(path)
    epochs = 100
    batch_size = 1024

    cnn_vae = train_cnn_vae(train, val, epochs=epochs, batch_size=batch_size, beta=i)

    #cnn_vae.save(f'{path}/full_cnn_vae_beta{i}.keras')
    tmp = CNN_VAE(eta_b=eta_b, phi_b=phi_b, beta=i
            )
    tmp.set_weights(cnn_vae.get_weights())
    tmp.save(f'{path}/full_cnn_vae_beta{i}.keras')


Epoch 1/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 83s 12ms/step - kl_loss: 40.0657 - loss: 3.5691 - reconstruction_loss: 3.5325 - val_kl_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_total_loss: 0.0000e+00
Epoch 2/100
  11/6569 ━━━━━━━━━━━━━━━━━━━━ 36s 6ms/step - kl_loss: 33.2840 - loss: 1.4031 - reconstruction_loss: 1.3712    

/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: kl_loss,loss,reconstruction_loss,val_kl_loss,val_reconstruction_loss,val_total_loss
  current = self.get_monitor_value(logs)


6569/6569 ━━━━━━━━━━━━━━━━━━━━ 59s 9ms/step - kl_loss: 32.8342 - loss: 1.3798 - reconstruction_loss: 1.3483 - val_kl_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_total_loss: 0.0000e+00
Epoch 3/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 60s 9ms/step - kl_loss: 30.6684 - loss: 1.3562 - reconstruction_loss: 1.3268 - val_kl_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_total_loss: 0.0000e+00
Epoch 4/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 59s 9ms/step - kl_loss: 29.0733 - loss: 1.3378 - reconstruction_loss: 1.3100 - val_kl_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_total_loss: 0.0000e+00
Epoch 5/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 59s 9ms/step - kl_loss: 30.9631 - loss: 0.9796 - reconstruction_loss: 0.9496 - val_kl_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_total_loss: 0.0000e+00
Epoch 6/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 59s 9ms/step - kl_loss: 31.1319 - loss: 0.8620 - reconstruction_loss: 0.8317 - val_kl_loss: 0.0000e+00 - val_reconstr

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_api.py:107: UserWarning: You are saving a model that has not yet been built. It might not contain any weights yet. Consider building the model first by calling it on some data.
  return saving_lib.save_model(model, filepath)


Epoch 1/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 66s 10ms/step - kl_loss: 16.4385 - loss: 3.5213 - reconstruction_loss: 3.3908 - val_kl_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_total_loss: 0.0000e+00
Epoch 2/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 59s 9ms/step - kl_loss: 12.5173 - loss: 1.4854 - reconstruction_loss: 1.3739 - val_kl_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_total_loss: 0.0000e+00
Epoch 3/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 61s 9ms/step - kl_loss: 14.2082 - loss: 1.1972 - reconstruction_loss: 1.0658 - val_kl_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_total_loss: 0.0000e+00
Epoch 4/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 61s 9ms/step - kl_loss: 15.7020 - loss: 0.9583 - reconstruction_loss: 0.8094 - val_kl_loss: 0.0000e+00 - val_reconstruction_loss: 0.0000e+00 - val_total_loss: 0.0000e+00
Epoch 5/100
6569/6569 ━━━━━━━━━━━━━━━━━━━━ 61s 9ms/step - kl_loss: 15.6890 - loss: 0.9250 - reconstruction_loss: 0.7759 - val_kl_loss: 0.0000e+00 -

# Results

In [ ]:
from sklearn.metrics import roc_curve, auc

    